In [1]:
# Import the image, math and os libraries
import PIL
from PIL import Image
import math
from math import radians, cos, sin, asin, sqrt
from os import listdir
from os.path import isfile, join# Loop over the elevation and satellite image set


In [2]:
# Import the image, math and os libraries

x_tile_range =[0,3]
y_tile_range = [0,4]


for img_name in ['vector','satellite']:   # Make a list of the image names 
#for img_name in ['satellite']:   # Make a list of the image names 
    print(img_name)
    
    image_files = ['./' + img_name + '_images/' + f for f in listdir('./' + img_name + '_images/')]    # Open the image set using pillow
    

    
    images = [PIL.Image.open(x) for x in image_files]   # Calculate the number of image tiles in each direction

    
    edge_length_x = x_tile_range[1] - x_tile_range[0]
    edge_length_y = y_tile_range[1] - y_tile_range[0]
    
    edge_length_x = max(1,edge_length_x)
    edge_length_y = max(1,edge_length_y)   # Find the final composed image dimensions  
    
    
    
    width, height = images[0].size
    
    total_width = width*edge_length_x
    
    total_height = height*edge_length_y   # Create a new blank image we will fill in

    composite = PIL.Image.new('RGB', (total_width, total_height))   # Loop over the x and y ranges

    y_offset = 0

    for i in range(0,edge_length_x):
        x_offset = 0

        for j in range(0,edge_length_y):  

            
            # Open up the image file and paste it into the composed image at the given offset position
            
            tmp_img = PIL.Image.open('./' + img_name + '_images/' + str(i) + '.' + str(j) + '.png')
            
            composite.paste(tmp_img, (y_offset,x_offset))
            
            x_offset += width # Update the width
        
        y_offset += height # Update the height
            
    # Save the final image
    filename = './composite_images/'+img_name+'.png'
    print("Filename:", filename)
    composite.save('./composite_images/'+img_name+'.png')
    print("Image Size:", PIL.Image.open(filename).size)
    print("\n")

vector
Filename: ./composite_images/vector.png
Image Size: (1536, 2048)


satellite
Filename: ./composite_images/satellite.png
Image Size: (1536, 2048)




In [3]:
print("Image Size:", PIL.Image.open(filename).size)

Image Size: (1536, 2048)


In [4]:
import mercantile

lat_lng = [51.999635, -0.455226] #location of mote
delta = 0.009 # scan range in degrees - about 1 km in all directions

tl = [lat_lng[0]+delta, lat_lng[1]-delta] # top left of image
br = [lat_lng[0]-delta, lat_lng[1]+delta] # bottom right of image
z = 15 # set the resolution (max at 15)

tl_tile = mercantile.tile(tl[1],tl[0],z) # returns a mercantile Tile Class
br_tile = mercantile.tile(br[1],br[0],z)

tl_coords = mercantile.ul(tl_tile)
br_coords = mercantile.ul( mercantile.Tile(br_tile.x+1,br_tile.y+1,z) )

In [5]:
print("Image top left coordinates:", tl_coords.lng, tl_coords.lat)
print("Image bottom right coordinates:", br_coords.lng, br_coords.lat)

Image top left coordinates: -0.472412109375 52.01193653675362
Image bottom right coordinates: -0.439453125 51.984880139916626


In [6]:
TL = (tl_coords.lng, tl_coords.lat)
TR = (br_coords.lng, tl_coords.lat)
BR = (br_coords.lng, br_coords.lat)


In [7]:

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 * 1000 # Radius of earth in meters. Use 3956 for miles
    return c * r


In [9]:
width = haversine(TL[0],TL[1],TR[0],TR[1])
height = haversine(TR[0],TR[1],BR[0],BR[1])

width_px, height_px = PIL.Image.open(filename).size
x_scale = width / width_px
y_scale = height / height_px

width_deg = TR[0] - TL[0]
height_deg = TR[1] - BR[1]

x_scale_deg = width_deg / width_px
y_scale_deg = height_deg / height_px

print("Image width (px):", width_px)
print("Image height (px):", height_px)
print("Image width (m):",width)
print("Image heiht (m):",height)
print("X scaling factor (m/px):", x_scale)
print("Y scaling factor (m/px):", y_scale)
print("Image width (deg):",width_deg)
print("Image height (deg):",height_deg)
print("X scaling factor (deg/px):", x_scale_deg)
print("Y scaling factor (deg/px):", y_scale_deg)


Image width (px): 1536
Image height (px): 2048
Image width (m): 2255.7186879104743
Image heiht (m): 3008.534061556077
X scaling factor (m/px): 1.4685668541083816
Y scaling factor (m/px): 1.4690107722441783
Image width (deg): 0.032958984375
Image height (deg): 0.02705639683699701
X scaling factor (deg/px): 2.1457672119140625e-05
Y scaling factor (deg/px): 1.3211131268064946e-05
